In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# check to see if GPU
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!nvidia-smi

import sys, os, distutils.core

!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

import numpy as np
import math, random

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

from PIL import Image
import cv2
from google.colab.patches import cv2_imshow

import os, json, pickle

import torch
import detectron2

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.utils.logger import setup_logger
setup_logger()

!nvcc --version
TORCH_VERSION = (torch.__version__)
print("torch: ", TORCH_VERSION)
print("detectron2:", detectron2.__version__)

/bin/bash: line 1: nvidia-smi: command not found
Cloning into 'detectron2'...
remote: Enumerating objects: 15912, done.
remote: Total 15912 (delta 0), reused 0 (delta 0), pack-reused 15912 (from 1)
Receiving objects: 100% (15912/15912), 6.67 MiB | 16.69 MiB/s, done.
Resolving deltas: 100% (11332/11332), done.


W1003 14:04:38.186000 1210 torch/utils/cpp_extension.py:118] No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.7 MB/s eta 0:00:00
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=c5fa4939bb814416ce3c8fce97e37aee1e3bd7d872bcfdaf30a8bba9b68173a0
  Stored in directory: /root/.cache/pip/wheels/ed/9f/a5/e4f5b27454ccd4596bd8b62432c7d6b1ca9fa22aef9d70a16a
Successfully built fvcore
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
torch:  2.8.0+cu126
detectron2: 0.6


In [3]:
#@title load images
from detectron2.data.datasets import register_coco_instances
# base_path = "/content/drive/MyDrive/DLforPVFacades/paper/dataset/4_dataset_final_PVandNonPV/PVFinderPaper.v3-pvandnonpv.coco-segmentation/"
base_path = "/content/drive/MyDrive/DLforPVFacades/paper/dataset/5_dataset_final_PVandNonPV_ColorAug/PVFinderPaper.v7-pvandnonpv_augflipcropcolor.coco-segmentation"

for split in ["test"]:
    try:
        MetadataCatalog.remove(split)
    except:
        pass
    try:
        DatasetCatalog.remove(split)
    except:
        pass

    register_coco_instances(split, {}, os.path.join(base_path, split, "_annotations_case2.coco.json"), os.path.join(base_path, split))
    print(f"{split} with {len([item for item in os.listdir(os.path.join(base_path, split)) if item.endswith('.jpg')])}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DLforPVFacades/paper/dataset/5_dataset_final_PVandNonPV_ColorAug/PVFinderPaper.v7-pvandnonpv_augflipcropcolor.coco-segmentation/test'

In [ ]:
import pandas as pd
import requests
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from io import BytesIO
import json
import os
import re
import pprint
from detectron2.utils.visualizer import ColorMode
from detectron2 import structures
from pycocotools import mask as mask_utils
import csv
import gc
from tqdm.notebook import tqdm

In [ ]:
pp = pprint.PrettyPrinter()

csv_file = '/content/drive/MyDrive/DLforPVFacades/paper/human_labeling/Batch_5301916_batch_results_fin.csv'
df = pd.read_csv(csv_file)

output_folder = '/content/drive/MyDrive/DLforPVFacades/paper/human_labeling/output'
os.makedirs(output_folder, exist_ok=True)

masks_folder = '/content/drive/MyDrive/DLforPVFacades/paper/human_labeling/masks'
os.makedirs(masks_folder, exist_ok=True)

dataset = 'test'
dataset_dicts = DatasetCatalog.get(dataset)
metadata = MetadataCatalog.get(dataset)

gt_dict = {}
for item in dataset_dicts:
    img_name = os.path.basename(item["file_name"]).split(".rf")[0]
    gt_dict[img_name] = item

results = []
for image_url, group in df.groupby('Input.image_url'):
    print(f"\n\n\n>>> img: {image_url}")


    '''
    LOAD GROUND TRUTH IMAGE
    '''
    img_name = image_url.split('.rf')[0]
    gt = gt_dict[img_name]

    print(f"h: {gt['height']}\tw:{gt['width']}")
    print(len(gt['annotations']))

    img = cv2.imread(gt["file_name"])

    # CREATE UNIFIED GT MASK
    gt_mask = None
    gt_union_mask = np.zeros_like(img[:, :, 0], dtype=int)
    for item in gt['annotations']:
        gt_mask = mask_utils.decode(item['segmentation']).astype(int)
        gt_union_mask = np.logical_or(gt_union_mask, gt_mask).astype(int)
    gt_mask = gt_union_mask

    '''
    BRING EACH WORKER'S ANNOTATION MASK (ANNOT)
    '''
    annotations = {}

    # for each worker, extract annotations
    for idx, row in group.iterrows():
        if row['AssignmentStatus'] != 'Approved':
            print(f"\n\n\nSKIPPING {row}")
            continue
        worker_id = row['WorkerId']
        polygons_data = row['Answer.annotatedResult.polygons']
        if pd.isna(polygons_data) or polygons_data == '[]':  # No annotation case
            annotations[worker_id] = None
        else:
            annotations[worker_id] = json.loads(polygons_data)



    for worker, polygons in annotations.items():
        polygons_vertices = []

        print(worker, polygons)
        if polygons is not None:
            for polygon in polygons:
                vertices = [(vertex['x'], vertex['y']) for vertex in polygon['vertices']]
                polygons_vertices.append(vertices)  # <-- This was wrongly indented

        pred_mask = np.zeros_like(img[:, :, 0], dtype=np.uint8)  # should be uint8 for cv2
        for polygon_vertices in polygons_vertices:
            pts = np.array([polygon_vertices], dtype=np.int32)  # must be wrapped in list
            cv2.fillPoly(pred_mask, pts, color=1)

        # Define the save directory
        img_folder_name = os.path.splitext(os.path.basename(image_url))[0]  # Remove .jpg
        save_dir = os.path.join(masks_folder, img_folder_name)

        # Create the directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # Define the path for this worker's mask
        mask_filename = f"{worker}.png"  # or .jpg if you prefer
        mask_path = os.path.join(save_dir, mask_filename)

        print()

        # Save the mask
        cv2.imwrite(mask_path, (pred_mask * 255).astype(np.uint8))  # Save mask as binary image (0 or 255)

        # Convert BGR to RGB for Matplotlib
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).copy()

        # Create base transparent background by dimming the image
        transparent_bg = (img_rgb * 0.2).astype(np.uint8)  # 20% brightness

        # Create overlay to add masks
        overlay = transparent_bg.copy()

        # Define colors
        gt_color = (255, 0, 0)          # Red
        pred_color = (255, 165, 0)      # Orange
        overlap_color = (0, 255, 0)     # Green

        # Overlay masks
        overlay[gt_mask == 1] = gt_color
        overlay[pred_mask == 1] = pred_color
        overlay[(gt_mask == 1) & (pred_mask == 1)] = overlap_color

        # Blend with the transparent base (which is already dimmed, so blending is optional)
        blended = overlay

        # Show both images side by side
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        axes[0].imshow(img_rgb)
        axes[0].set_title("Original Image")
        axes[0].axis('off')

        axes[1].imshow(blended)
        axes[1].set_title(f"Worker {worker} vs GT")
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

        # DETECTION-LEVEL METRICS
        gt_detected = gt_mask.sum() > 0
        pred_detected = pred_mask.sum() > 0

        if gt_detected and pred_detected:
            det_category = 'TP'
        elif not gt_detected and pred_detected:
            det_category = 'FP'
        elif gt_detected and not pred_detected:
            det_category = 'FN'
        else:
            det_category = 'TN'

        # Calculate PIXEL-LEVEL TP, FP, FN, TN
        tp = np.logical_and(gt_mask == 1, pred_mask == 1).sum()
        fp = np.logical_and(gt_mask == 0, pred_mask == 1).sum()
        fn = np.logical_and(gt_mask == 1, pred_mask == 0).sum()
        tn = np.logical_and(gt_mask == 0, pred_mask == 0).sum()

        # Optionally calculate metrics
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        iou = tp / (tp + fp + fn)

        print(f"\nMetrics for Worker {worker}:")
        print(f"TP: {tp}, FP: {fp}, FN: {fn}, TN: {tn}")
        print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}, IoU: {iou:.3f}")

        # Inside loop per image & worker:
        results.append({
            'image': image_url,
            'worker': worker,
            'det_category': det_category,
            'img_precision': precision,
            'img_recall': recall,
            'img_f1': f1,
            'img_iou': iou,
            'TP_pixels': tp,
            'FP_pixels': fp,
            'FN_pixels': fn,
            'TN_pixels': tn,
            })

# After the full loop:
metrics_df = pd.DataFrame(results)
# metrics_df.to_csv('/content/drive/MyDrive/DLforPVFacades/paper/human_labeling/HumanLabelingEvaluation.csv', index=False)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
metrics_df.to_csv('/content/drive/MyDrive/DLforPVFacades/paper/human_labeling/humanVSai.csv', index=False)

In [ ]:
metrics_df = pd.read_csv('/content/drive/MyDrive/DLforPVFacades/paper/human_labeling/humanVSai.csv')

In [ ]:
metrics_df

,image,worker,total_TP,total_FP,total_FN,total_TN,total_Prec,total_Rec,total_F1,det_TP,det_FP,det_FN,det_TN,det_Prec,det_Rec,det_F1
0,013_gsv_00_jpg.rf.8239ca8fd1e9b4c8602c307b3c50...,ARTQWWNFA7IOD,15826,2164,0,391610,0.879711,1.000000,0.936007,1,0,0,0,1.0,1.0,1.0
1,013_gsv_00_jpg.rf.8239ca8fd1e9b4c8602c307b3c50...,A1IZ4NX41GKU4X,0,0,15826,393774,0.000000,0.000000,0.000000,0,0,1,0,0.0,0.0,0.0
2,013_gsv_00_jpg.rf.8239ca8fd1e9b4c8602c307b3c50...,A3QZMGTVA4VO44,15012,138,814,393636,0.990891,0.948566,0.969267,1,0,0,0,1.0,1.0,1.0
3,013_gsv_00_jpg.rf.8239ca8fd1e9b4c8602c307b3c50...,A7V4CVENA0DYV,15826,1198,0,392576,0.929629,1.000000,0.963531,1,0,0,0,1.0,1.0,1.0
4,013_gsv_00_jpg.rf.8239ca8fd1e9b4c8602c307b3c50...,A234WZ4KX8IW07,15826,929,0,392845,0.944554,1.000000,0.971486,1,0,0,0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,93_jpg.rf.f70be11a1a6715ae40845ab07e49e5a8.jpg,A1IZ4NX41GKU4X,0,0,0,1465000,0.000000,0.000000,0.000000,0,0,0,1,0.0,0.0,0.0
543,93_jpg.rf.f70be11a1a6715ae40845ab07e49e5a8.jpg,ABVM2KJ7CRNZ0,0,20732,0,1444268,0.000000,0.000000,0.000000,0,1,0,0,0.0,0.0,0.0
544,93_jpg.rf.f70be11a1a6715ae40845ab07e49e5a8.jpg,A7V4CVENA0DYV,0,0,0,1465000,0.000000,0.000000,0.000000,0,0,0,1,0.0,0.0,0.0
545,93_jpg.rf.f70be11a1a6715ae40845ab07e49e5a8.jpg,A3AA5G6HENO6VJ,0,0,0,1465000,0.000000,0.000000,0.000000,0,0,0,1,0.0,0.0,0.0


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=metrics_df)

https://docs.google.com/spreadsheets/d/1Rb_0FiEVKY28E-Iytuqr4wtmrR0VmW49tRyZR-60WlY/edit#gid=0


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=metrics_df)

https://docs.google.com/spreadsheets/d/1S2ZHlRLx83PJ9qNgni-6qZ8oMMTlEwjbdER2pMXt4xU/edit#gid=0


### grouped by worker

In [ ]:
# Group by worker and sum TP, FP, FN, TN
worker_grouped = metrics_df.groupby('worker').agg({
    'total_TP': 'sum',
    'total_FP': 'sum',
    'total_FN': 'sum',
    'total_TN': 'sum',
    'det_TP': 'sum',
    'det_FP': 'sum',
    'det_FN': 'sum',
    'det_TN': 'sum'
}).reset_index()

# Count number of images each worker annotated
image_counts = metrics_df.groupby('worker').size().reset_index(name='num_images')

# Merge counts into the grouped summary
worker_grouped = worker_grouped.merge(image_counts, on='worker')

# Define function to compute precision, recall, F1 for both pixel-level and detection-level
def compute_metrics(row):
    tp, fp, fn = row['total_TP'], row['total_FP'], row['total_FN']
    det_tp, det_fp, det_fn = row['det_TP'], row['det_FP'], row['det_FN']

    # Pixel-level metrics
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    # Detection-level metrics
    det_precision = det_tp / (det_tp + det_fp) if (det_tp + det_fp) > 0 else 0
    det_recall = det_tp / (det_tp + det_fn) if (det_tp + det_fn) > 0 else 0
    det_f1 = 2 * det_precision * det_recall / (det_precision + det_recall) if (det_precision + det_recall) > 0 else 0

    return pd.Series({
        'Pixel_Precision': precision,
        'Pixel_Recall': recall,
        'Pixel_F1': f1,
        'Det_Precision': det_precision,
        'Det_Recall': det_recall,
        'Det_F1': det_f1
    })

# Apply metric computation to each row
metrics_summary = worker_grouped.join(worker_grouped.apply(compute_metrics, axis=1))

# Save the summary to CSV
metrics_summary.to_csv('/content/drive/MyDrive/DLforPVFacades/paper/human_labeling/humanVSai.csv', index=False)

In [ ]:
metrics_summary

,worker,total_TP,total_FP,total_FN,total_TN,det_TP,det_FP,det_FN,det_TN,num_images,Pixel_Precision,Pixel_Recall,Pixel_F1,Det_Precision,Det_Recall,Det_F1
0,A10NF5TK6IFNX6,32606,2730,249010,1727996,3,0,0,0,3,0.922742,0.115782,0.205747,1.000000,1.000000,1.000000
1,A14W0AXTJ3R19V,0,0,600090,8378352,0,0,4,5,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,A16DOX0X2E9XBI,0,0,449046,3685120,0,0,3,4,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,A16X5FB3HAFCKN,750656,383703,433075,13914457,6,2,4,8,20,0.661745,0.634144,0.647650,0.750000,0.600000,0.666667
4,A1E6RS45GUAFC3,23624,10621,493,124062,1,0,0,0,1,0.689853,0.979558,0.809568,1.000000,1.000000,1.000000
5,A1IZ4NX41GKU4X,272676,65273,4732640,64671604,5,1,31,50,87,0.806855,0.054477,0.102063,0.833333,0.138889,0.238095
6,A1XO6ONCCTBMKW,1058623,357246,391318,8966179,5,3,2,6,16,0.747684,0.730115,0.738795,0.625000,0.714286,0.666667
7,A1YYDRHVY4LL8K,356163,392654,413049,6070612,5,2,1,1,9,0.475634,0.463023,0.469244,0.714286,0.833333,0.769231
8,A234WZ4KX8IW07,5158278,763293,336139,28842794,31,2,0,0,33,0.871100,0.938822,0.903694,0.939394,1.000000,0.968750
9,A272A0JGRTBFCR,1903764,571645,389421,18853641,20,2,1,7,30,0.769070,0.830183,0.798459,0.909091,0.952381,0.930233


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=metrics_summary)

https://docs.google.com/spreadsheets/d/1r5kZe79tcBAq_8IPoE3W50UZLtvqp2hCuX9DphzwVwo/edit#gid=0


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=metrics_summary)

https://docs.google.com/spreadsheets/d/1hiouyIrVM5GkGdf-2T6LlgG363rDzTc5rj46w1fpQPY/edit#gid=0


### grouped by image, best worker results

In [ ]:
# Make sure your results are in a DataFrame
best_metrics_df = metrics_df.copy()

# Split into two parts
has_gt = best_metrics_df[best_metrics_df['total_TP'] + best_metrics_df['total_FN'] > 0]
no_gt = best_metrics_df[best_metrics_df['total_TP'] + best_metrics_df['total_FN'] == 0]

# --- Part 1: Images with GT mask ---
best_with_gt = (
    has_gt.sort_values(by='total_F1', ascending=False)
          .drop_duplicates(subset='image', keep='first')
)

# --- Part 2: Images with no GT mask (pure TN) ---
# First prefer workers who didn't annotate (perfect TN), then fallback to lowest FP
def pick_best_for_no_gt(group):
    tn_only = group[(group['total_FP'] == 0)]  # ideally all TN
    if not tn_only.empty:
        return tn_only.iloc[0]
    else:
        return group.sort_values(by='total_FP').iloc[0]

best_no_gt = no_gt.groupby('image').apply(pick_best_for_no_gt).reset_index(drop=True)

# Combine both
best_per_image = pd.concat([best_with_gt, best_no_gt], ignore_index=True)

# Save to CSV
best_per_image.to_csv('/content/drive/MyDrive/DLforPVFacades/paper/human_labeling/best_worker_per_image.csv', index=False)

KeyError: 'total_F1'

In [ ]:
best_per_image

,image,worker,total_TP,total_FP,total_FN,total_TN,total_Prec,total_Rec,total_F1,det_TP,det_FP,det_FN,det_TN,det_Prec,det_Rec,det_F1
0,133_gsv_00_jpg.rf.d65f791083d8eb7b2d8abd388996...,A3QZMGTVA4VO44,15943,340,101,393216,0.979119,0.993705,0.986358,1,0,0,0,1.0,1.0,1.0
1,098_gsv_00_jpg.rf.fbd55a0d6a6496bacc825087addc...,A3QZMGTVA4VO44,41954,634,535,366477,0.985113,0.987409,0.986260,1,0,0,0,1.0,1.0,1.0
2,161_web_00_jpg.rf.af092d7246707aa7a012c7bfbd27...,A3QZMGTVA4VO44,53469,738,843,544950,0.986386,0.984479,0.985431,1,0,0,0,1.0,1.0,1.0
3,240_web_01_jpg.rf.f5f0d78ac430efe7e349ee214b16...,ABVM2KJ7CRNZ0,136699,3832,581,458888,0.972732,0.995768,0.984115,1,0,0,0,1.0,1.0,1.0
4,271_web_02_jpg.rf.0cd9f533b758985329a52f6c9ed6...,A272A0JGRTBFCR,406217,12031,2855,1519657,0.971235,0.993021,0.982007,1,0,0,0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,62_jpg.rf.cbcb1284731131484ca93e50ea39f327.jpg,A1IZ4NX41GKU4X,0,0,0,392000,0.000000,0.000000,0.000000,0,0,0,1,0.0,0.0,0.0
101,74_jpg.rf.133448adda1d567c71a86896659167d0.jpg,A1IZ4NX41GKU4X,0,0,0,450000,0.000000,0.000000,0.000000,0,0,0,1,0.0,0.0,0.0
102,78_142728_328nopv_jpg.rf.62eac90f9a0ce6650bf2e...,A3AA5G6HENO6VJ,0,0,0,409600,0.000000,0.000000,0.000000,0,0,0,1,0.0,0.0,0.0
103,89_jpg.rf.2e4229a434fc489540a1da7bafca76c7.jpg,A2RVSBEHUCNC6K,0,0,0,50325,0.000000,0.000000,0.000000,0,0,0,1,0.0,0.0,0.0


In [ ]:
# Sum pixel-based and detection-based confusion matrix components
total_tp = best_per_image['total_TP'].sum()
total_fp = best_per_image['total_FP'].sum()
total_fn = best_per_image['total_FN'].sum()
total_tn = best_per_image['total_TN'].sum()

det_tp = best_per_image['det_TP'].sum()
det_fp = best_per_image['det_FP'].sum()
det_fn = best_per_image['det_FN'].sum()
det_tn = best_per_image['det_TN'].sum()

# Recalculate pixel-level metrics
pixel_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
pixel_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
pixel_f1 = 2 * pixel_precision * pixel_recall / (pixel_precision + pixel_recall) if (pixel_precision + pixel_recall) > 0 else 0

# Recalculate detection-level metrics
det_precision = det_tp / (det_tp + det_fp) if (det_tp + det_fp) > 0 else 0
det_recall = det_tp / (det_tp + det_fn) if (det_tp + det_fn) > 0 else 0
det_f1 = 2 * det_precision * det_recall / (det_precision + det_recall) if (det_precision + det_recall) > 0 else 0

# Print the summary
print("=== Pixel-Level Summary ===")
print(f"TP: {total_tp}, FP: {total_fp}, FN: {total_fn}, TN: {total_tn}")
print(f"Precision: {pixel_precision:.3f}, Recall: {pixel_recall:.3f}, F1: {pixel_f1:.3f}")

print("\n=== Detection-Level Summary ===")
print(f"TP: {det_tp}, FP: {det_fp}, FN: {det_fn}, TN: {det_tn}")
print(f"Precision: {det_precision:.3f}, Recall: {det_recall:.3f}, F1: {det_f1:.3f}")

=== Pixel-Level Summary ===
TP: 6543992, FP: 906720, FN: 1310810, TN: 76447675
Precision: 0.878, Recall: 0.833, F1: 0.855

=== Detection-Level Summary ===
TP: 50, FP: 0, FN: 3, TN: 52
Precision: 1.000, Recall: 0.943, F1: 0.971


### grouped by image, average worker results

In [ ]:
grouped_img = metrics_df.copy()

# Step 1: Average per image (across all workers)
per_image_avg = grouped_img.groupby('image').agg({
    'total_TP': 'mean',
    'total_FP': 'mean',
    'total_FN': 'mean',
    'total_TN': 'mean',
    'det_TP': 'mean',
    'det_FP': 'mean',
    'det_FN': 'mean',
    'det_TN': 'mean'
}).reset_index()

# Step 2: Sum the averaged values across images
total_tp = per_image_avg['total_TP'].sum()
total_fp = per_image_avg['total_FP'].sum()
total_fn = per_image_avg['total_FN'].sum()
total_tn = per_image_avg['total_TN'].sum()

det_tp = per_image_avg['det_TP'].sum()
det_fp = per_image_avg['det_FP'].sum()
det_fn = per_image_avg['det_FN'].sum()
det_tn = per_image_avg['det_TN'].sum()

# Step 3: Recalculate precision, recall, F1
pixel_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
pixel_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
pixel_f1 = 2 * pixel_precision * pixel_recall / (pixel_precision + pixel_recall) if (pixel_precision + pixel_recall) > 0 else 0

det_precision = det_tp / (det_tp + det_fp) if (det_tp + det_fp) > 0 else 0
det_recall = det_tp / (det_tp + det_fn) if (det_tp + det_fn) > 0 else 0
det_f1 = 2 * det_precision * det_recall / (det_precision + det_recall) if (det_precision + det_recall) > 0 else 0

# Step 4: Print
print("=== Pixel-Level Summary (Avg per Image) ===")
print(f"TP: {total_tp:.2f}, FP: {total_fp:.2f}, FN: {total_fn:.2f}, TN: {total_tn:.2f}")
print(f"Precision: {pixel_precision:.3f}, Recall: {pixel_recall:.3f}, F1: {pixel_f1:.3f}")

print("\n=== Detection-Level Summary (Avg per Image) ===")
print(f"TP: {det_tp:.2f}, FP: {det_fp:.2f}, FN: {det_fn:.2f}, TN: {det_tn:.2f}")
print(f"Precision: {det_precision:.3f}, Recall: {det_recall:.3f}, F1: {det_f1:.3f}")

=== Pixel-Level Summary (Avg per Image) ===
TP: 3869319.47, FP: 2124098.23, FN: 3985482.53, TN: 75230296.77
Precision: 0.646, Recall: 0.493, F1: 0.559

=== Detection-Level Summary (Avg per Image) ===
TP: 32.57, FP: 5.97, FN: 20.43, TN: 46.03
Precision: 0.845, Recall: 0.614, F1: 0.712


In [ ]:
per_image_avg

,image,total_TP,total_FP,total_FN,total_TN,det_TP,det_FP,det_FN,det_TN
0,013_gsv_00_jpg.rf.8239ca8fd1e9b4c8602c307b3c50...,12498.000000,885.8,3328.000000,392888.2,0.8,0.0,0.2,0.0
1,014_gsv_00_jpg.rf.77ba19eafadda66e595e6e5e2d03...,7311.833333,39042.5,14652.166667,348593.5,0.5,0.0,0.5,0.0
2,014_web_00_jpg.rf.a1a8b04b70168eea1c54286091ed...,7347.400000,6514.2,11718.600000,408819.8,1.0,0.0,0.0,0.0
3,021_web_00_jpg.rf.b99a633673138ed4ecebeb45876c...,47964.200000,41006.8,2294.800000,368392.2,1.0,0.0,0.0,0.0
4,035_web_00_jpg.rf.d74c7e9b8b50701addc672ab754b...,6728.400000,3200.0,11627.600000,336881.0,0.4,0.0,0.6,0.0
...,...,...,...,...,...,...,...,...,...
100,62_jpg.rf.cbcb1284731131484ca93e50ea39f327.jpg,0.000000,31506.8,0.000000,360493.2,0.0,0.4,0.0,0.6
101,74_jpg.rf.133448adda1d567c71a86896659167d0.jpg,0.000000,0.0,0.000000,450000.0,0.0,0.0,0.0,1.0
102,78_142728_328nopv_jpg.rf.62eac90f9a0ce6650bf2e...,0.000000,0.0,0.000000,409600.0,0.0,0.0,0.0,1.0
103,89_jpg.rf.2e4229a434fc489540a1da7bafca76c7.jpg,0.000000,0.0,0.000000,50325.0,0.0,0.0,0.0,1.0
